# Machine Learning baseline

In [9]:
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from fastcore.basics import Path, AttrDict
import utils
import numpy as np
import pickle
import os
import sys

module_paths = [
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('.')),
]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)


# This is used to import the evaluation script, not needed for training
import sys
sys.path.append('../') 
import evaluation
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)


In [10]:
config = AttrDict(
    # challenge_data_dir = Path('../../dataset/data_subset/'),
    challenge_data_dir = Path('../../dataset/phase_1_v3/'),
    valid_ratio = .5,
    lag_steps = 0,
    tolerance= 6, # Default evaluation tolerance
)

In [11]:
# Define the list of feature columns
delta_column = "Delta_SemimajorAxis"
inclination_diff = "Inclination Diff"

feature_cols = [
    "Eccentricity",
    "Semimajor Axis (m)",
    "Inclination (deg)",
    "RAAN (deg)",
    "Argument of Periapsis (deg)",
    "True Anomaly (deg)",
    # "Latitude (deg)",
    "Longitude (deg)",
    "Altitude (m)",
    # "X (m)",
    # "Y (m)",
    # "Z (m)",
    # "Vx (m/s)",
    # "Vy (m/s)",
    # "Vz (m/s)",
    # delta_column,
    # inclination_diff,
]

In [12]:
# Define the directory paths
train_data_dir = config.challenge_data_dir / "train"

# Load the ground truth data
ground_truth = pd.read_csv(config.challenge_data_dir / 'train_labels.csv')

# Apply the function to the ground truth data
data, updated_feature_cols = utils.tabularize_data(train_data_dir, feature_cols, 
                                          ground_truth, lag_steps=config.lag_steps)

# For each ObjectID, show the first rows of the columns TimeIndex, ObjectID, EW, and NS
# data[['ObjectID', 'TimeIndex' , 'EW', 'NS']].groupby('ObjectID').head(2).head(10)
data.head()

,Timestamp,Eccentricity,Semimajor Axis (m),Inclination (deg),RAAN (deg),Argument of Periapsis (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (m),...,Vx (m/s),Vy (m/s),Vz (m/s),ObjectID,TimeIndex,Delta_SemimajorAxis,Inclination Diff,Inclination Direction,EW,NS
0,2022-09-01 00:00:00.000000Z,0.000202,4.216537e+07,0.139822,94.427336,52.733092,277.810980,-0.014460,85.119506,3.578607e+07,...,-2786.228658,1300.258746,6.534142,1,0,0.000000,0.000000,-1,SS-HK,SS-HK
1,2022-09-01 02:00:00.000000Z,0.000214,4.216458e+07,0.140008,94.407240,59.443909,301.205730,-0.007812,85.122842,3.578177e+07,...,-3062.958599,-271.603808,7.513546,1,1,0.000000,0.000187,1,SS-HK,SS-HK
2,2022-09-01 04:00:00.000000Z,0.000231,4.216450e+07,0.140139,94.420500,64.355257,326.372111,0.001030,85.131695,3.577825e+07,...,-2514.294041,-1770.701547,6.465207,1,2,-870.131537,0.000131,1,SS-HK,SS-HK
3,2022-09-01 06:00:00.000000Z,0.000255,4.216524e+07,0.140147,94.419710,67.664873,353.158326,0.009633,85.144601,3.577642e+07,...,-1287.802531,-2792.783114,3.667072,1,3,653.280794,0.000008,1,SS-HK,SS-HK
4,2022-09-01 08:00:00.000000Z,0.000277,4.216620e+07,0.140112,94.348243,70.366450,20.624466,0.015595,85.158717,3.577713e+07,...,285.868898,-3062.068198,-0.129326,1,4,1707.623420,-0.000035,-1,SS-HK,SS-HK


In [13]:
# Create a validation set without mixing the ObjectIDs
object_ids = data['ObjectID'].unique()
train_ids, valid_ids = train_test_split(object_ids, 
                                        test_size=config.valid_ratio)

train_data = data[data['ObjectID'].isin(train_ids)].copy()
valid_data = data[data['ObjectID'].isin(valid_ids)].copy()

ground_truth_train = ground_truth[ground_truth['ObjectID'].isin(train_ids)].copy()
ground_truth_valid = ground_truth[ground_truth['ObjectID'].isin(valid_ids)].copy()

# Count the number of objects in the training and validation sets
print('Number of objects in the training set:', len(train_data['ObjectID'].unique()))
print('Number of objects in the validation set:', len(valid_data['ObjectID'].unique()))

Number of objects in the training set: 950
Number of objects in the validation set: 950


Next we will make sure that there every label, both in the direction EW and NS,
is present both in the training and validation partitions

In [14]:
# Get the unique values of EW and NS in train and test data
train_EW = set(train_data['EW'].unique())
train_NS = set(train_data['NS'].unique())
valid_EW = set(valid_data['EW'].unique())
valid_NS = set(valid_data['NS'].unique())

# Get the values of EW and NS that are in test data but not in train data
missing_EW = valid_EW.difference(train_EW)
missing_NS = valid_NS.difference(train_NS)

# Check if all the values in EW are also present in NS
if not set(train_data['EW'].unique()).issubset(set(train_data['NS'].unique())):
    # Get the values of EW that are not present in NS
    missing_EW_NS = set(train_data['EW'].unique()).difference(
        set(train_data['NS'].unique())
    )
else:
    missing_EW_NS = None

# Print the missing values of EW and NS
print("Missing values of EW in test data:", missing_EW)
print("Missing values of NS in test data:", missing_NS)
print("Values of EW not present in NS:", missing_EW_NS)


Missing values of EW in test data: set()
Missing values of NS in test data: set()
Values of EW not present in NS: {'AD-NK'}


In [15]:
# Convert categorical data to numerical data
le_EW = LabelEncoder()
le_NS = LabelEncoder()

# Encode the 'EW' and 'NS' columns
train_data['EW_encoded'] = le_EW.fit_transform(train_data['EW'])
train_data['NS_encoded'] = le_NS.fit_transform(train_data['NS'])
valid_data['Predicted_EW'] = 'ES-ES'

print("Columns encoded")

# Define the Random Forest model for EW
# model_EW = RandomForestClassifier(n_estimators=200, random_state=42)
# Define the Random Forest model for NS
model_NS = RandomForestClassifier(n_estimators=100)

model_NS.fit(train_data[updated_feature_cols], train_data['NS_encoded'])

Path('trained_model').mkdir(exist_ok=True)
pickle.dump(model_NS, open('trained_model/model_NS_v3.pkl', 'wb'))
pickle.dump(le_NS, open('trained_model/le_NS_v3.pkl', 'wb'))

Columns encoded


MemoryError: Unable to allocate 58.6 GiB for an array with shape (3808, 2064504) and data type float64

In [7]:
import matplotlib.pyplot as plt
def plot_kbest_score(min, max, score_list, title):
    fig, ax = plt.subplots()
    x = np.arange(min, max+1)
    y = score_list

    ax.bar(x,y,width = 0.2)
    ax.set_xlabel('Number of features selected')
    ax.set_ylabel(title)
    ax.set_ylim(0,1.2)
    ax.set_xticks(np.arange(min,max+1))
    ax.set_xticklabels(np.arange(min, max+1), fontsize=12)

    for i, v in enumerate(y):
        plt.text(x=i+min, y=v+0.05, s=str(v), ha='center')

    plt.tight_layout()
    plt.show()

In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import ns_id_Validator
import ns_ik_Validator
import ew_Validator
import ss_Validator

# TRAINED_MODEL_DIR = Path('./trained_model/')

# # Load the trained models (don't use the utils module, use pickle)
# model_EW = pickle.load(open(TRAINED_MODEL_DIR / 'model_EW_v3.pkl', 'rb'))
# le_EW = pickle.load(open(TRAINED_MODEL_DIR / 'le_EW_v3.pkl', 'rb'))
# model_NS = pickle.load(open(TRAINED_MODEL_DIR / 'model_NS_v3.pkl', 'rb'))
# le_NS = pickle.load(open(TRAINED_MODEL_DIR / 'le_NS_v3.pkl', 'rb'))

# Convert categorical data to numerical data
le_EW = LabelEncoder()
le_NS = LabelEncoder()

# Encode the 'EW' and 'NS' columns
train_data['EW_encoded'] = le_EW.fit_transform(train_data['EW'])
train_data['NS_encoded'] = le_NS.fit_transform(train_data['NS'])
valid_data['Predicted_EW'] = 'ES-ES'

print("Columns encoded")

# Define the Random Forest model for EW
# model_EW = RandomForestClassifier(n_estimators=200, random_state=42)
# Define the Random Forest model for NS
model_NS = RandomForestClassifier(n_estimators=200, random_state=42)

f2_score_list = []
recall_list = []
precision_list = []
ss_validator = ss_Validator.SS_Validator()
ns_ik_validator = ns_ik_Validator.NS_IK_Validator()
ns_id_validator = ns_id_Validator.NS_ID_Validator()

k_min = 3
k_max = 8
for k in range(k_min,k_max+1):
    selector = SelectKBest(mutual_info_classif, k=k)
    selector.fit(train_data[updated_feature_cols], train_data['NS_encoded'])
    selector_mask = selector.get_support()
    print(train_data[updated_feature_cols].columns[selector_mask])

    sel_X_train = selector.transform(train_data[updated_feature_cols])
    sel_X_test = selector.transform(valid_data[updated_feature_cols])


    # Fit the model to the training data for NS
    model_NS.fit(sel_X_train, train_data['NS_encoded'])

    print("NS Classifier is Fit for k=" +str(k))

    # Make predictions on the training data for NS
    valid_data['Predicted_NS'] = le_NS.inverse_transform(
        model_NS.predict(sel_X_test)
    )

    test_results = utils.convert_classifier_output(valid_data)
    validated_results = ss_validator.apply_validator(test_results, valid_data)

    validated_results = ns_ik_validator.apply_validator(validated_results, valid_data)

    # validated_results = ns_id_validator.apply_validator(validated_results, valid_data)

    evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, validated_results, 
                                                tolerance=config.tolerance)
    precision, recall, f2, rmse = evaluator.score_NS(debug=True)
    f2_score_list.append(round(f2, 3))
    print("F2 Score: " + str(round(f2, 3)))
    recall_list.append(round(recall, 3))
    print("Recall: " + str(round(recall, 3)))
    precision_list.append(round(precision,3))
    print("Precision: " + str(round(precision, 3)))
    print("RMSE: " + str(round(rmse, 3)))


plot_kbest_score(k_min, k_max, f2_score_list, "F2 Score")
plot_kbest_score(k_min, k_max, recall_list, "Recall")
plot_kbest_score(k_min, k_max, precision_list, "Precision")
    
Path('trained_model').mkdir(exist_ok=True)
pickle.dump(model_NS, open('trained_model/model_NS_v3.pkl', 'wb'))
pickle.dump(le_NS, open('trained_model/le_NS_v3.pkl', 'wb'))

Columns encoded


MemoryError: Unable to allocate 58.7 GiB for an array with shape (3816, 2064492) and data type float64

In [ ]:
test_results = utils.convert_classifier_output(valid_data)
validated_results = ss_validator.apply_validator(test_results, valid_data)

print("Before EW Validator:")
evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, validated_results, 
                                            tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score_EW(debug=True)
f2_score_list.append(round(f2, 3))
print("F2 Score: " + str(round(f2, 3)))
recall_list.append(round(recall, 3))
print("Recall: " + str(round(recall, 3)))
precision_list.append(round(precision,3))
print("Precision: " + str(round(precision, 3)))
print("RMSE: " + str(round(rmse, 3)))


print("After EW Validator:")
validated_results = ew_validator.apply_validator(validated_results, valid_data)

evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, validated_results, 
                                            tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score_EW(debug=True)
f2_score_list.append(round(f2, 3))
print("F2 Score: " + str(round(f2, 3)))
recall_list.append(round(recall, 3))
print("Recall: " + str(round(recall, 3)))
precision_list.append(round(precision,3))
print("Precision: " + str(round(precision, 3)))
print("RMSE: " + str(round(rmse, 3)))

In [ ]:
selector = SelectKBest(mutual_info_classif, k=6) # set the best k
selector.fit(train_data[updated_feature_cols], train_data['EW_encoded'])
selected_feature_mask = selector.get_support()

selected_features = train_data[updated_feature_cols].columns[selected_feature_mask]
selected_features

In [ ]:
def apply_ss_validator(results, data):
        mask = (results['Node']=='SS')&(results['TimeIndex']!=0)
        stripped_results = results[~mask]
        merged_results = stripped_results.copy()
        merged_results = merged_results.sort_values(by=['ObjectID', 'TimeIndex']).reset_index(drop=True)
        return merged_results

DELTA_COLUMN = "Delta_SemimajorAxis"
MANEUVER_THRESHOLD = 3000
def apply_ew_maneuver_validator(results, data):
        validate_results = results[(results['Node']!='SS') & (results['Direction']=='EW')]
        to_drop = []
        for result_index, result in validate_results.iterrows():
            event_data = data[(data['ObjectID']==result['ObjectID']) & (data['TimeIndex']>(result['TimeIndex']-5)) & (data['TimeIndex']<(result['TimeIndex']+5))]
            #TODO : Should I move the event to the data point?
            fits_threshold = (event_data[DELTA_COLUMN].abs()>MANEUVER_THRESHOLD).any()
            if not fits_threshold:
                to_drop.append(result_index)
                
        for index in to_drop:
            results = results.drop(index)
        return results

INCLINATION_DIR = "Inclination Direction"
def apply_ns_ik_maneuver_validator(results, data):
        validate_results = results[(results['Node']=='IK') & (results['Direction']=='NS') ]
        to_drop = []
        for result_index, result in validate_results.iterrows():
            event_data = data[(data['ObjectID']==result['ObjectID']) & (data['TimeIndex']>(result['TimeIndex']-5)) & (data['TimeIndex']<(result['TimeIndex']+5))]
            #TODO : Should I move the event to the data point?
            validated = True
            direction_sum = event_data[INCLINATION_DIR].sum()
            if (direction_sum>=8) | (direction_sum<=-8):
                validated = False
            if not validated:
                to_drop.append(result_index)
                
        for index in to_drop:
            results = results.drop(index)
        return results

In [ ]:
def apply_ns_id_maneuver_validator(results, data):
    mask = (results['Node']=='ID') & (results['Direction']=='NS')
    validated_results = results[~mask].copy()
    return validated_results

The `NodeDetectionEvaluator` class in the evaluation module allows not only to
compute the general score for a given dataset, but get evaluations per object, and
even plots that show how the predictions look like in a timeline

In [ ]:



train_results = utils.convert_classifier_output(train_data)

print("ML Results: "+str(train_results.shape[0]))
evaluator = evaluation.NodeDetectionEvaluator(ground_truth_train, train_results, 
                                              tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score(debug=True)
print(f'Precision for the train set: {precision:.2f}')
print(f'Recall for the train set: {recall:.2f}')
print(f'F2 for the train set: {f2:.2f}')
print(f'RMSE for the train set: {rmse:.2f}')

# ss_validator = SSValidator()
validated_results = apply_ss_validator(train_results, data)

print("")
print("SS Validated: "+str(validated_results.shape[0]))
evaluator = evaluation.NodeDetectionEvaluator(ground_truth_train, validated_results, 
                                              tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score(debug=True)
print(f'Precision for the train set: {precision:.2f}')
print(f'Recall for the train set: {recall:.2f}')
print(f'F2 for the train set: {f2:.2f}')
print(f'RMSE for the train set: {rmse:.2f}')

# maneuver_validator = ManeuverValidator()
validated_results = apply_ew_maneuver_validator(validated_results, data)

print("")
print("EW Validated: "+str(validated_results.shape[0]))

evaluator = evaluation.NodeDetectionEvaluator(ground_truth_train, validated_results, 
                                              tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score(debug=True)
print(f'Precision for the train set: {precision:.2f}')
print(f'Recall for the train set: {recall:.2f}')
print(f'F2 for the train set: {f2:.2f}')
print(f'RMSE for the train set: {rmse:.2f}')

# maneuver_validator = ManeuverValidator()
validated_results = apply_ns_ik_maneuver_validator(validated_results, data)

print("")
print("NS-IK Validated: "+str(validated_results.shape[0]))

evaluator = evaluation.NodeDetectionEvaluator(ground_truth_train, validated_results, 
                                              tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score(debug=True)
print(f'Precision for the train set: {precision:.2f}')
print(f'Recall for the train set: {recall:.2f}')
print(f'F2 for the train set: {f2:.2f}')
print(f'RMSE for the train set: {rmse:.2f}')


validated_results = apply_ns_id_maneuver_validator(validated_results, data)
print("")
print("NS-ID Validated: "+str(validated_results.shape[0]))

evaluator = evaluation.NodeDetectionEvaluator(ground_truth_train, validated_results, 
                                              tolerance=config.tolerance)
precision, recall, f2, rmse = evaluator.score(debug=True)
print(f'Precision for the train set: {precision:.2f}')
print(f'Recall for the train set: {recall:.2f}')
print(f'F2 for the train set: {f2:.2f}')
print(f'RMSE for the train set: {rmse:.2f}')


In [ ]:
# Plot the evaluation timeline for a random ObjectID from the training set
evaluator.plot(np.random.choice(train_data['ObjectID'].unique()))

In [ ]:
# Loop over the Object IDs in the training set and call the evaluation
# function for each object and aggregate the results
total_tp = 0
total_fp = 0
total_fn = 0
total_wn = 0
total_wt = 0
total_nm = 0
for oid in train_data['ObjectID'].unique():
    tp, fp, fn, gt_object, p_object, wrong_nodes, wrong_type, not_matched, ew_fp, ns_fp,ns_ik_fp, ns_id_fp = evaluator.evaluate(oid)
    total_tp += tp
    total_fp += fp
    total_fn += fn
    total_wn += wrong_nodes
    total_wt += wrong_type
    total_nm += not_matched

print(f'Total true positives: {total_tp}')
print(f'Total false positives: {total_fp}')
print(f'Total false negatives: {total_fn}')
print(f'Total wrong nodes: {total_wn}')
print(f'Total wrong types: {total_wt}')
print(f'Total not matched: {total_nm}')

In [ ]:

if config.valid_ratio > 0:
    valid_results = utils.convert_classifier_output(valid_data)
    print("ML Results: "+str(valid_results.shape[0]))
    evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, 
                                                  valid_results,
                                                  tolerance=config.tolerance)
    precision, recall, f2, rmse = evaluator.score(debug=True)
    print(f'Precision for the validation set: {precision:.2f}')
    print(f'Recall for the validation set: {recall:.2f}')
    print(f'F2 for the validation set: {f2:.2f}')
    print(f'RMSE for the validation set: {rmse:.2f}')

    

    # ss_validator = SSValidator()
    validated_results = apply_ss_validator(valid_results, data)

    print("")
    print("SS Validated: "+str(validated_results.shape[0]))
    evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, 
                                                  validated_results,
                                                  tolerance=config.tolerance)
    precision, recall, f2, rmse = evaluator.score(debug=True)
    print(f'Precision for the validation set: {precision:.2f}')
    print(f'Recall for the validation set: {recall:.2f}')
    print(f'F2 for the validation set: {f2:.2f}')
    print(f'RMSE for the validation set: {rmse:.2f}')

    # maneuver_validator = ManeuverValidator()
    validated_results = apply_ew_maneuver_validator(validated_results, data)
    
    print("")
    print("EW Validated: "+ str(validated_results.shape[0]))
    evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, 
                                                  validated_results,
                                                  tolerance=config.tolerance)
    precision, recall, f2, rmse = evaluator.score(debug=True)
    print(f'Precision for the validation set: {precision:.2f}')
    print(f'Recall for the validation set: {recall:.2f}')
    print(f'F2 for the validation set: {f2:.2f}')
    print(f'RMSE for the validation set: {rmse:.2f}')

    validated_results = apply_ns_ik_maneuver_validator(validated_results, data)
    
    print("")
    print("NS-IK Validated: "+ str(validated_results.shape[0]))
    evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, 
                                                  validated_results,
                                                  tolerance=config.tolerance)
    precision, recall, f2, rmse = evaluator.score(debug=True)
    print(f'Precision for the validation set: {precision:.2f}')
    print(f'Recall for the validation set: {recall:.2f}')
    print(f'F2 for the validation set: {f2:.2f}')
    print(f'RMSE for the validation set: {rmse:.2f}')


    validated_results = apply_ns_id_maneuver_validator(validated_results, data)
    print("")
    print("NS-ID Validated: "+ str(validated_results.shape[0]))
    evaluator = evaluation.NodeDetectionEvaluator(ground_truth_valid, 
                                                  validated_results,
                                                  tolerance=config.tolerance)
    precision, recall, f2, rmse = evaluator.score(debug=True)
    print(f'Precision for the validation set: {precision:.2f}')
    print(f'Recall for the validation set: {recall:.2f}')
    print(f'F2 for the validation set: {f2:.2f}')
    print(f'RMSE for the validation set: {rmse:.2f}')
# validated_results

In [ ]:
# Plot the evaluation timeline for a random ObjectID from the training set
evaluator.plot(np.random.choice(valid_data['ObjectID'].unique()))

In [ ]:
# Save the trained random forest models (and label encoders) to disk
# Create the folder trained_model if it doesn't exist
Path('trained_model').mkdir(exist_ok=True)
pickle.dump(model_EW, open('trained_model/model_EW_v3.pkl', 'wb'))
pickle.dump(model_NS, open('trained_model/model_NS_v3.pkl', 'wb'))
pickle.dump(le_EW, open('trained_model/le_EW_v3.pkl', 'wb'))
pickle.dump(le_NS, open('trained_model/le_NS_v3.pkl', 'wb'))